In [1]:
import fitz  # PyMuPDF
import re
import spacy
import pymupdf
import string
import pdfplumber
import pandas as pd

In [20]:
nlp = spacy.load('en_core_web_trf')
pdf_path= r'C:\Users\arafe\Downloads\Glossary_Of_Risk_Management.pdf'

In [28]:
def extract_text_pdfplumber(pdf_path):
    text = ""
    try:
        # Open the PDF
        pdf_document = fitz.open(pdf_path)
        
        # Iterate through the pages
        for page_num in range(0, 8):  # Customize the page range as per your needs
            page = pdf_document.load_page(page_num)  # Get a page
            text += page.get_text("text")  # Extract text from the page, format: "text"
        
        pdf_document.close()  # Close the document after extraction
    except Exception as e:
        print(f"Error reading the PDF file: {e}")
    
    return text

# Example usage with spaCy
doc = nlp(extract_text_pdfplumber(pdf_path))

In [29]:
text =extract_text_pdfplumber(pdf_path)

In [30]:
text

'1\nAccepted Deliverables. Products, results, or capabili\xad\nties produced by a project and validated by the pro\xad\nject customer or sponsors as meeting their specified \nacceptance criteria.\nAccuracy. An assessment of correctness within the \nquality management system.\nAcquisition. Obtaining human and material resources \nnecessary to perform project activities. Acquisition im\xad\nplies a cost of resources and is not necessarily financial.\nActivity. A distinct, scheduled portion of work perfor\xad\nmed during the course of a project. \nActivity Cost Estimates. The projected cost of the plan\xad\nned activity, which includes the cost of all resources re\xad\nquired to execute and complete the activity, including \nall cost components and types.\nActivity Duration. The time in calendar units between \nthe start and finish of a schedule activity.\nActivity List. A documented tabulation of schedule ac\xad\ntivities that shows the activity description, activity iden\xad\ntifier, an

In [33]:
import re

def extract_sections(text):
    sections = []
    # Regex pattern to match bold-like sections
    section_pattern = re.compile(r'^\s*([A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*)\.\s')

    # Split the text into lines
    lines = text.split('\n')
    
    # Iterate through each line, capturing the line and its index
    for idx, line in enumerate(lines):
        # Check if the line matches the section pattern
        match = section_pattern.match(line)
        if match:
            sections.append((idx, match.group(1)))  # Append the index and the section title

    return sections


In [35]:
keywords = extract_sections(text)
print(keywords)

[(1, 'Accepted Deliverables'), (5, 'Accuracy'), (7, 'Acquisition'), (10, 'Activity'), (12, 'Activity Cost Estimates'), (16, 'Activity Duration'), (18, 'Activity List'), (25, 'Agreements'), (30, 'Alternatives Analysis'), (33, 'Alternatives Generation'), (40, 'Assumption'), (43, 'Assumptions Analysis'), (51, 'Benchmarking'), (56, 'Brainstorming'), (60, 'Budget'), (66, 'Data Date'), (68, 'Data Gathering Techniques'), (71, 'Deliverable'), (74, 'Document Analysis'), (77, 'Documentation Reviews'), (80, 'Change Request'), (82, 'Constraint'), (84, 'Contingency'), (87, 'Contingency Reserve'), (91, 'Contract'), (95, 'Control Limits'), (99, 'Corrective Action'), (108, 'Criteria'), (111, 'Critical Path'), (114, 'Earned Value Management'), (117, 'Enterprise Environmental Factors'), (120, 'Estimate Activity Durations'), (123, 'Estimate Activity Resources'), (129, 'Critical Path Activity'), (131, 'Customer'), (136, 'Facilitated Workshops'), (140, 'Forecast'), (148, 'Free Float'), (152, 'Guideline'), 

In [37]:
lines = text.split('\n')
lines[469:477]

['Specification. A document that completely, accura\xad',
 'tely, and verifiably expresses the requirements, de\xad',
 'sign, behavior, and other characteristics of a system, ',
 'component, product, result, or service, as well as the ',
 'procedures to determine if these provisions have been ',
 'met. Some examples are requirements specifications, ',
 'design specifications, product specifications, and test ',
 'specifications.']

In [38]:
import pandas as pd
data_frame_of_sections = pd.DataFrame()
section=[]
ids=[]
for i in range(len (keywords)):
    section.append(keywords[i][1])
    ids.append(keywords[i][0])
data_frame_of_sections['Keywords'] = section
data_frame_of_sections['Ids'] = ids
data_frame_of_sections.head()


,Keywords,Ids
0,Accepted Deliverables,1
1,Accuracy,5
2,Acquisition,7
3,Activity,10
4,Activity Cost Estimates,12


In [45]:
def extract_paragraph__(text,start_idx,end_edx):
    lines = text.split('\n')
    paragraph=lines[start_idx:end_edx]
    combined_string = ' '.join(paragraph)
    return combined_string

In [46]:
def extract_paragraphs_to_dataframe(df, text):
    """
    Extracts paragraphs between consecutive section IDs in the DataFrame and injects the results.
    
    :param df: A DataFrame containing section IDs.
    :param text: The complete text as a string.
    :return: The updated DataFrame with extracted paragraphs.
    """
    paragraphs = []  # List to store extracted paragraphs

    # Iterate through the DataFrame rows to extract paragraphs
    for i in range(len(df) - 1):
        start_id = df.loc[i, 'Ids']  # Current ID
        end_id = df.loc[i + 1, 'Ids']  # Next ID
        print(start_id,end_id)
        # Extract the paragraph between the two IDs
        paragraph = extract_paragraph__(text, start_id, end_id)
        if paragraph=="":
            paragraph= "no_paragraph"
        paragraphs.append(paragraph)  # Append extracted paragraph to the list

    # Add the extracted paragraphs to a new column in the DataFrame
    # Fill with None or empty string for the last ID
    paragraphs.append("no_paragraph")  # Append empty string for the last ID

    df['paragraphs'] = paragraphs  # Create new column 'paragraphs'

    return df

In [47]:
df=extract_paragraphs_to_dataframe(data_frame_of_sections, text)
df.head()
# data_frame_of_sections.to_csv('DataSet.csv', index=False)

1 5
5 7
7 10
10 12
12 16
16 18
18 25
25 30
30 33
33 40
40 43
43 51
51 56
56 60
60 66
66 68
68 71
71 74
74 77
77 80
80 82
82 84
84 87
87 91
91 95
95 99
99 108
108 111
111 114
114 117
117 120
120 123
123 129
129 131
131 136
136 140
140 148
148 152
152 155
155 158
158 160
160 167
167 170
170 175
175 178
178 181
181 184
184 190
190 192
192 196
196 200
200 204
204 207
207 211
211 213
213 216
216 220
220 223
223 225
225 228
228 232
232 236
236 239
239 245
245 249
249 253
253 267
267 269
269 272
272 275
275 279
279 283
283 287
287 289
289 294
294 298
298 301
301 306
306 308
308 311
311 315
315 325
325 328
328 331
331 336
336 341
341 344
344 347
347 351
351 354
354 358
358 365
365 368
368 370
370 373
373 377
377 380
380 385
385 390
390 392
392 394
394 398
398 405
405 408
408 411
411 414
414 416
416 421
421 427
427 430
430 434
434 437
437 440
440 442
442 447
447 449
449 452
452 455
455 458
458 463
463 465
465 467
467 469
469 477
477 480
480 484
484 487
487 491
491 497
497 501
501 504
504 507
50

,Keywords,Ids,paragraphs
0,Accepted Deliverables,1,"Accepted Deliverables. Products, results, or c..."
1,Accuracy,5,Accuracy. An assessment of correctness within ...
2,Acquisition,7,Acquisition. Obtaining human and material reso...
3,Activity,10,"Activity. A distinct, scheduled portion of wor..."
4,Activity Cost Estimates,12,Activity Cost Estimates. The projected cost of...


In [48]:
data_frame_of_sections.to_csv('DataSet_GRM.csv', index=False)